In [1]:
import itertools
import gc
from bib import *
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import backend as K 
from keras import  activations
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Dense, Activation, regularizers,LSTM,Bidirectional,CuDNNLSTM
import tensorflow as tf
from  tqdm import tqdm_notebook
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [3]:
xtrains,ytrains,xvals,yvals,xtest,ytest=timeSplit(15,('CHL 1','THERM 1','SSH','SR','WS'),
                                               ['CHL '+ str(i) for i in range(2,19)])
ytrains,yvals,ytest=applyToy(lambda x:np.log(x)/np.log(10),ytrains,yvals,ytest)

In [4]:
def generate():
    layers=[]
    layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                                merge_mode='ave',input_shape=(15,5)))
    layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                                merge_mode='ave',input_shape=(15,5)))
    layers.append(Dense(17,activation='linear',kernel_regularizer=regularizers.l2(0.01)))
    model = Sequential(layers)
    model.compile(optimizer='rmsprop',
              loss='mse')
    return model
callbacks = [EarlyStopping(monitor='val_loss', patience=150)]

In [5]:
def validate(xtrains,ytrains,xvals,yvals):
    fold_evaluation=[]
    for xt,yt,xv,yv in tqdm_notebook(zip(xtrains,ytrains,xvals,yvals),total=len(xtrains)):
        model=generate()
        
#         mean=xt.mean()
#         std=xt.std()
        
#         xt=(xt-mean)/std
#         xv=(xv-mean)/std
        
        
        yt=np.exp(yt*np.log(10))
        yv=np.exp(yv*np.log(10))        
        
        model.fit(xt,yt,
                  epochs=10000000,
                  callbacks=callbacks,
                  verbose=0,
                  batch_size=73*10, 
                  validation_data=(xv,yv))
        
        yp = model.predict(xv)
        #check error in original space
        
#         yp=np.exp(yp*np.log(10))
#         yv=np.exp(yv*np.log(10))       
        fold_evaluation.append(evaluate(yp.reshape(-1),yv.reshape(-1)))
        print(evaluate(yp.reshape(-1),yv.reshape(-1)))
        
    return np.mean(fold_evaluation),np.std(fold_evaluation )


In [ ]:
K.clear_session()
layers=[]
layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                            merge_mode='ave',input_shape=(15,5)))
layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                            merge_mode='ave',input_shape=(15,5)))
layers.append(Dense(17,activation='linear',kernel_regularizer=regularizers.l2(0.01)))
model = Sequential(layers)
model.compile(optimizer='rmsprop',
          loss='mse')


model.fit(xtrains[0],np.exp(ytrains[0]*np.log(10)),
          epochs=1000000,
          callbacks=callbacks,
          verbose=1,
          batch_size=73*10, 
          validation_data=(xvals[0],np.exp(yvals[0]*np.log(10))))

Train on 10386 samples, validate on 59 samples
Epoch 1/1000000
10386/10386 [==============================] - 5s 446us/step - loss: 26.1431 - val_loss: 2.3579
Epoch 2/1000000
10386/10386 [==============================] - 1s 137us/step - loss: 1.5536 - val_loss: 1.1781
Epoch 3/1000000
10386/10386 [==============================] - 2s 145us/step - loss: 0.8657 - val_loss: 0.7526
Epoch 4/1000000
10386/10386 [==============================] - 1s 140us/step - loss: 0.6133 - val_loss: 0.6724
Epoch 5/1000000
10386/10386 [==============================] - 1s 141us/step - loss: 0.4551 - val_loss: 0.4469
Epoch 6/1000000
10386/10386 [==============================] - 1s 141us/step - loss: 0.3946 - val_loss: 0.3915
Epoch 7/1000000
10386/10386 [==============================] - 1s 143us/step - loss: 0.3540 - val_loss: 0.3077
Epoch 8/1000000
10386/10386 [==============================] - 1s 142us/step - loss: 0.3018 - val_loss: 0.2754
Epoch 9/1000000
10386/10386 [==============================] - 1

In [135]:

K.clear_session()

# print('ACT :'+ str(act))
error,std=validate(xtrains,ytrains,xvals,yvals)
print('\t error: %.5f +- %.5f' % (error,std))

0.03537967972126437
0.029332094699235613
0.02681153639675254
0.031998671731821825
0.023703929995749953
0.04129674024291694
0.03024409678219249
0.029993975262374382
0.02717887702399664
0.03466402257410357
0.025688827280629586
0.027392033286911928
0.027345969949634497
0.023297350338352125
0.033531461366533746
0.04237775949804656
	 error: 0.03064 +- 0.00546


In [ ]:
error: 0.06898 +- 0.01279
    
150 patience
0.03537967972126437
0.029332094699235613
0.02681153639675254
0.031998671731821825
0.023703929995749953
0.04129674024291694
0.03024409678219249
0.029993975262374382
0.02717887702399664
0.03466402257410357
0.025688827280629586
0.027392033286911928
0.027345969949634497
0.023297350338352125
0.033531461366533746
0.04237775949804656
	 error: 0.03064 +- 0.00546